# average work hours

Load the required libraries and specify a credentials file (downloaded from https://developers.google.com/calendar/quickstart/python) after enabling the api: 

In [34]:
import datetime
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
from dateutil import parser

SCOPES = 'https://www.googleapis.com/auth/calendar.readonly'

creds = 'credentials.json'

This function uses the credentials file in the current directory to authenticate with the GCal API and build a calendar service that we can use to query events. 

In [45]:
def authenticate(credsfile):
    token = file.Storage('token.json')
    creds = token.get()
    
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets(credsfile, SCOPES)
        creds = tools.run_flow(flow, store)
    
    service = build('calendar', 'v3', http=creds.authorize(Http()))
    
    return service 

In [102]:
service = authenticate(creds)
query = 'entered work'
query2 = 'exited work'

Get the events created when I enter and exit work from the service

In [103]:
enter_events = service.events().list(calendarId='primary', singleEvents='True', q=query).execute()
exit_events = service.events().list(calendarId='primary', singleEvents='True',q=query2).execute()

In [105]:
enter_results = enter_events.get('items', [])
exit_results = exit_events.get('items', [])

Convert the list of event objects to a list of datetime objects using dateutil

In [106]:
def to_datetime_list(results):
    l = []
    for event in results:
        start = event['start'].get('dateTime')
        start = str(start)
        start = parser.parse(start)
        l.append(start)
    return l

Clean up duplicates, set output formatting (verified manually, need to add error checking).  

In [133]:
entered = to_datetime_list(enter_results)
exited = to_datetime_list(exit_results)

del exited[2]
del entered[9]

formatting = '%I:%M %p %d'

print(len(entered))
print(len(exited))

20
19


Iterate through exited and entered list and sum the total time spent at work, divide by number of days worked. 

In [142]:
total_time = datetime.timedelta()

for time in entered: 
    print(time.strftime(formatting))
print(" ")
for time in exited:
    print(time.strftime(formatting))
print(" ")
for i in range(len(exited)):
    delta = exited[i]-entered[i]
    total_time += delta
    print(delta)
print(" ")
print(total_time/15)

07:30 AM 26
01:01 PM 26
07:43 AM 27
07:46 AM 28
07:37 AM 31
01:32 PM 31
07:35 AM 02
08:12 AM 03
07:43 AM 04
07:44 AM 07
08:19 AM 08
07:34 AM 09
07:51 AM 10
07:52 AM 11
07:55 AM 14
12:37 PM 14
07:36 AM 15
01:00 PM 15
08:03 AM 16
01:45 PM 16
 
01:00 PM 26
04:06 PM 26
04:16 PM 27
04:05 PM 28
12:31 PM 31
03:07 PM 31
04:38 PM 02
05:19 PM 03
02:32 PM 04
05:58 PM 07
05:05 PM 08
06:03 PM 09
04:31 PM 10
04:44 PM 11
12:18 PM 14
04:57 PM 14
11:56 AM 15
07:36 PM 15
11:46 AM 16
 
5:30:00
3:05:00
8:33:00
8:19:00
4:54:00
1:35:00
9:03:00
9:07:00
6:49:00
10:14:00
8:46:00
10:29:00
8:40:00
8:52:00
4:23:00
4:20:00
4:20:00
6:36:00
3:43:00
 
8:29:12


datetime.timedelta(days=-1, seconds=66600)